In [1]:
N = 3
values = [[0] * (i+1) for i in range(N)]

# Assigning profit of each item
values[0][0] = 60 
values[1][0] = 30
values[1][1] = 100   
values[2][0] = 2  
values[2][1] = 5  
values[2][2] = 120  

# Assigning weight of each item
weights = [10, 20, 30]

W_capacity = 50

In [2]:
import dimod
import dimod.utilities

# https://docs.ocean.dwavesys.com/en/latest/docs_dimod/reference/generated/dimod.binary.BinaryQuadraticModel.to_ising.html

x = [dimod.Binary(i) for i in range(N)]

cost = - dimod.quicksum(values[i][j] * x[i] * x[j] for i in range(N) for j in range (i+1)) # values
cost -= 0.9603*(W_capacity - dimod.quicksum(weights[i] * x[i] for i in range(N)))
cost += 0.0371*(W_capacity - dimod.quicksum(weights[i] * x[i] for i in range(N)))**2

model = dimod.BinaryQuadraticModel(cost.linear, cost.quadratic, cost.offset, vartype='BINARY')
H = model.to_ising()

In [3]:
from neal import SimulatedAnnealingSampler


sampleset = SimulatedAnnealingSampler().sample(model, num_reads=1000)

solution = sampleset.first.sample
print(solution)

{0: 1, 1: 1, 2: 1}


In [4]:
import numpy as np
from DMRG.annealing_ham_to_MPO import from_ham_coeff

h_coeffs = [H[0][site] for site in range (N)] # H[0] contains linear interactions

J_coeffs = np.zeros((N,N))
for term in H[1]: # quadratic interacions
    # TODO in formulation i>j, but in MPO i<j (triangular superior, la meitat buida)
    J_coeffs[min(term[0], term[1]), max(term[0], term[1])] = H[1][term]

offset = H[2]
s = 1

M = from_ham_coeff(N, J_coeffs, h_coeffs, s)

print(len(M))

3


In [5]:
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt

from ncon import ncon
from DMRG.module import doDMRG_MPO

##### Set bond dimensions and simulation options
chi = 16
Nsites = N

OPTS_numsweeps = 4 # number of DMRG sweeps
OPTS_dispon = 2 # level of output display
OPTS_updateon = True # level of output display
OPTS_maxit = 2 # iterations of Lanczos method
OPTS_krydim = 4 # dimension of Krylov subspace

#####################################

L_dim = M[0].shape[0]
R_dim = M[-1].shape[2]

# Dummy MPO boundary matrices
ML = np.array([1,0,0,0]).reshape(4,1,1) #left MPO boundary
MR = np.array([0,0,0,1]).reshape(4,1,1) #right MPO boundary

# Define the left MPO boundary
ML = np.zeros((L_dim, 1, 1))
ML[0, 0, 0] = 1

MR = np.zeros((R_dim, 1, 1))
MR[-1, 0, 0] = 1

#### Initialize MPS tensors
chid = 2 # local dimension
A = [0 for x in range(Nsites)]
A[0] = np.random.rand(1,chid,min(chi,chid))
for k in range(1,Nsites):
    A[k] = np.random.rand(A[k-1].shape[2],chid,min(min(chi,A[k-1].shape[2]*chid),chid**(Nsites-k-1)))

In [6]:
#### Do DMRG sweeps (2-site approach)
En1, left_MPS, sWeight, right_MPS = doDMRG_MPO(A,ML,M,MR,chi,
                                               numsweeps = OPTS_numsweeps,
                                               dispon = OPTS_dispon,
                                               updateon = OPTS_updateon,
                                               maxit = OPTS_maxit,
                                               krydim = OPTS_krydim)


Sweep: 1 of 4, Loc: 1,Energy: -163.055339
Sweep: 1 of 4, Loc: 0,Energy: -163.056000
Sweep: 1 of 4, Loc: 0,Energy: -163.056000
Sweep: 1 of 4, Loc: 1,Energy: -163.056000
Sweep: 2 of 4, Loc: 1,Energy: -163.056000
Sweep: 2 of 4, Loc: 0,Energy: -163.056000
Sweep: 2 of 4, Loc: 0,Energy: -163.056000
Sweep: 2 of 4, Loc: 1,Energy: -163.056000
Sweep: 3 of 4, Loc: 1,Energy: -163.056000
Sweep: 3 of 4, Loc: 0,Energy: -163.056000
Sweep: 3 of 4, Loc: 0,Energy: -163.056000
Sweep: 3 of 4, Loc: 1,Energy: -163.056000
Sweep: 4 of 4, Loc: 1,Energy: -163.056000
Sweep: 4 of 4, Loc: 0,Energy: -163.056000
Sweep: 4 of 4, Loc: 0,Energy: -163.056000
Sweep: 4 of 4, Loc: 1,Energy: -163.056000
